# Common Crawl Language Model - 30%

Big thanks to [Florian Leuerer](https://github.com/floleuerer) for his ULMFiT notebooks for German ([repo link](https://github.com/floleuerer/fastai_ulmfit_german)). They helped significantly!

We will be using the Common Crawl Mongolian corpus [CC-100](http://data.statmt.org/cc-100/) scraped from January-December 2018. 

This notebook uses only 30% of the dataset with a 20% validation holdout. 

## Download Dataset

## Library Installs and Imports

We will need several libraries to get started. Each should be installed and upgraded to ensure it works. This notebook was created in February 2020 and should work with the following major verisons:

- Fast.ai version 2.x
- Fastcore version: 1.x
- Pandas version: 1.x
- Numpy version: 1.x

In [1]:
!pip install -Uqq fastai --upgrade
!pip install -Uqq fastcore --upgrade
!pip install -Uqq pandas==1.1.0

In [2]:
import fastai
import fastcore
from fastai.text.all import *
from fastai.callback.progress import CSVLogger

print('Fast.ai version:', fastai.__version__)
print('Fastcore version:', fastcore.__version__)

Fast.ai version: 2.2.7
Fastcore version: 1.3.19


In [3]:
import pandas as pd
import numpy as np

print('Pandas version:', pd.__version__)
print('numpy version:', np.__version__)

Pandas version: 1.1.0
numpy version: 1.19.5


We will be using Google Drive to store our files. Please change the directory for the `cd` command to wherever your notebook is located. 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd '/content/drive/MyDrive/DataScience/NLP/02_mongolian_language_model'

/content/drive/MyDrive/DataScience/NLP/02_mongolian_language_model


## Create Data Loader for Language Model

We will be using data in a Pandas Dataframe to build our language model. However, you could also use txt files. 

In [6]:
df = pd.read_csv('data/mn.txt', header=None, names=['text'])

In [7]:
df.head(2)

,text
0,Түмэнбаярын Бум-Эрдэнэ: “Дотроосоо” - 2 (Өгүүллэг) # #
1,Маш их таалагдлаа


In [8]:
df = df.dropna()

In [9]:
df.shape

(13827747, 1)

Because of our large dataset, we will select 30% of our data to start with. 

In [10]:
np.random.seed(42)

In [11]:
df['filter'] = np.random.choice([0,1], size=len(df), p=[0.7, 0.3])

In [12]:
df = df[df['filter'] == 1].drop(columns=['filter'])

We will be using the ColSplitter, which expects a column called is_valid (with a boolean) to generate the validation set. We will randomly assign 10% to the validation set.

In [13]:
np.random.seed(42)

In [14]:
df['is_valid'] = np.random.choice([0,1], size=len(df), p=[0.8, 0.2])

We will set the batch size to 32. These can be changed depending on your GPU and RAM. 

The `is_lm` is set to `True` because we are training a language model. We are using a `get_x` but not a `get_y` because our target variable will be generated for us by the fastai library. 

In [15]:
bs = 32

In [16]:
dblock = DataBlock(blocks=TextBlock.from_df('text', is_lm=True),
                   get_x=ColReader('text'),
                   splitter=ColSplitter())

dls = dblock.dataloaders(df, bs=bs, seq_len=80)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Now you can see the `text_` column, which is our target variable. Our language model will be attempting to predict the next token in the sequence. 

In [17]:
dls.show_batch()

,text,text_
0,xxbos монголын шинэ үеийн түүх xxbos мөн хил орчмын аялал жуулчлалыг хөгжүүлэхэд агаарын шинэ зам нээх шаардлагатай байна гэдэг хүсэлтийг тавьсан бол xxunk ххк - ийн хувьд “ агаарын өндрийн хэмжээг журманд 6 xxrep 3 0 км гээд xxunk xxbos эмэгтэй нь дур xxbos уул уурхай - нийтлэл xxbos хөдөлмөр эрхлэлтийн үйлчилгээ xxbos xxunk - xxunk маш нөхөрсөг . хэрвээ танай гэр бүл ам бүл олонтой xxbos өмнөх мэдээ өдөр тутмын эдгээр дадал зуршлууд танил санагдаж байна уу ? тэгвэл,монголын шинэ үеийн түүх xxbos мөн хил орчмын аялал жуулчлалыг хөгжүүлэхэд агаарын шинэ зам нээх шаардлагатай байна гэдэг хүсэлтийг тавьсан бол xxunk ххк - ийн хувьд “ агаарын өндрийн хэмжээг журманд 6 xxrep 3 0 км гээд xxunk xxbos эмэгтэй нь дур xxbos уул уурхай - нийтлэл xxbos хөдөлмөр эрхлэлтийн үйлчилгээ xxbos xxunk - xxunk маш нөхөрсөг . хэрвээ танай гэр бүл ам бүл олонтой xxbos өмнөх мэдээ өдөр тутмын эдгээр дадал зуршлууд танил санагдаж байна уу ? тэгвэл үүнээс
1,% -иас дээш байх xxbos арван нэгдүгээр сар – ном худалдаанд гарна . xxbos баянгол xxup start xxup up “ гарааны бизнес эрхлэгч залуучуудын у … xxbos холбоос засах xxbos xxmaj lesbea milf нь асар их байгалийн xxmaj boobs нь зөөлөн дулаан үтрээний redtube.com xxbos үүний нэг бол хөгжлийн бэрхшээлтэй иргэдийн урласан бүтээлийн үзэсгэлэн . тус үзэсгэлэн соёлын төв өргөөнд 10:00 цагт нээлтээ хийсэн . xxbos нүүдэлчин монголчуудын эдийн соёлын нэг үндсэн хэсэг нь тэдний сууц xxbos энэтхэг улсад эрчим,-иас дээш байх xxbos арван нэгдүгээр сар – ном худалдаанд гарна . xxbos баянгол xxup start xxup up “ гарааны бизнес эрхлэгч залуучуудын у … xxbos холбоос засах xxbos xxmaj lesbea milf нь асар их байгалийн xxmaj boobs нь зөөлөн дулаан үтрээний redtube.com xxbos үүний нэг бол хөгжлийн бэрхшээлтэй иргэдийн урласан бүтээлийн үзэсгэлэн . тус үзэсгэлэн соёлын төв өргөөнд 10:00 цагт нээлтээ хийсэн . xxbos нүүдэлчин монголчуудын эдийн соёлын нэг үндсэн хэсэг нь тэдний сууц xxbos энэтхэг улсад эрчим хүчний


Our code below show the number of sequences of 80 tokens for our training and validation sets. 

In [18]:
len(dls.train), len(dls.valid)

(15389, 3859)

In [19]:
15389 * 80

492448

Here we can move to the language model training, or if the training is already compplete, move to below to test the model.

## Training The Language Model

In [21]:
notebook_path = Path('')

We will be using the AWD_LSTM architecture for our language model. We set the `pretrained` option to `False` as we will not be using a pretrained language model (that's what we are doing!).

In [22]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.5, pretrained=False, 
                               metrics=[accuracy, Perplexity()]).to_fp16()

In [23]:
learn.path = notebook_path.absolute()

Here we set our learning rate. Ideally we would find our ideal learning rate for our data. 

**Note:** *room for improvement*

In [24]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [25]:
cbs=[CSVLogger(fname='history_cc_lm.csv')]

In [26]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8, 0.7, 0.8), cbs=cbs)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.750010,4.661270,0.300710,105.770340,1:27:56
1,4.714205,4.590579,0.301251,98.551430,1:28:02
2,4.525063,4.423696,0.318295,83.403984,1:28:02
3,4.436898,4.315775,0.329929,74.871620,1:27:47
4,4.349831,4.223644,0.340383,68.281868,1:27:18
5,4.242636,4.131486,0.350137,62.270386,1:27:34
6,4.115372,4.038826,0.361377,56.759640,1:27:28
7,4.055038,3.959313,0.370508,52.421307,1:28:12


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.750010,4.661270,0.300710,105.770340,1:27:56
1,4.714205,4.590579,0.301251,98.551430,1:28:02
2,4.525063,4.423696,0.318295,83.403984,1:28:02
3,4.436898,4.315775,0.329929,74.871620,1:27:47
4,4.349831,4.223644,0.340383,68.281868,1:27:18
5,4.242636,4.131486,0.350137,62.270386,1:27:34
6,4.115372,4.038826,0.361377,56.759640,1:27:28
7,4.055038,3.959313,0.370508,52.421307,1:28:12
8,3.936707,3.906770,0.377134,49.738014,1:28:09
9,3.930506,3.895031,0.378853,49.157600,1:27:34


In [27]:
learn.to_fp32().save('mn_cc_03_lm', with_opt=False)

Path('/content/drive/My Drive/DataScience/NLP/02_mongolian_language_model/models/mn_cc_03_lm.pth')

In [28]:
with open('models/mn_cc_03_vocab.pkl', 'wb') as f:
      pickle.dump(learn.dls.vocab, f)

In [29]:
learn.save_encoder('mn_cc_03_lm_encoder')

## Test out the model

We won't expect the results to be very good, but we can at least see if the language model is learning reasonably decent. 

In [30]:
lm_fns = ['models/mn_cc_lm', 'models/mn_cc_vocab']

In [ ]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.5, pretrained=True, 
                               pretrained_fnames=lm_fns, model_dir='', metrics=[accuracy, Perplexity()]).to_fp16()

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/autocast_mode.py:114: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


In [38]:
TEXT = "Та аз жаргалтай байна уу?"

In [39]:
preds = []
N_WORDS = 50
N_SENTENCES = 1
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [40]:
print("\n".join(preds))

та аз жаргалтай байна уу ? ямар ч байсан нэг л зүйлийг мэдэхгүй хүмүүс үзээрэй холбоо барих / хууль зүйн акт / ажлын байрны тодорхойлолт мэдээллийн технологи .. монгол улсын хөгжлийн банк 2 . у.г и.доржсамбуу – а.а н.золбоо · спортын мэдээ : жүдо бөхийн дашт - д оролцох
